In [14]:
import numpy as np
import pytest


PRIME_64 = 18446744072637906947
PRIME_128 = 340282366920938463463374607429104828419
PRIME_256 = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF98C00003
PRIME = PRIME_64
PRIME_BITS = int(np.log2(PRIME))

print("PRIME BITS: ", PRIME_BITS)
print("PRIME: ", PRIME)

import random


def create_random_upper_triangular_matrix(n: int):
    """
    Create a random upper triangular matrix with the specified dimensions.

    Args:
        n (int): The size of the matrix.

    Returns:
        list: A list of lists representing a random upper triangular matrix.

    """
    L = PRIME >> 1
    matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i, n):
            matrix[i][j] = random.randint(-L, L - 1)
    return matrix


def create_random_lower_triangular_matrix(n: int):
    """
    Create a random lower triangular matrix with the specified dimensions.

    Args:
        n (int): The size of the matrix.

    Returns:
        list: A list of lists representing a random lower triangular matrix.

    """
    L = PRIME >> 1
    print(-L, L - 1)
    matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1):
            matrix[i][j] = random.randint(-L, L - 1)
        matrix[i][i] = 1  # Fill diagonal with 1s
    return matrix


def reveal_array(array: np.ndarray):
    """
    Recursively reveals the elements of the input array.

    Args:
        array (np.ndarray): The input array.

    Returns:
        np.ndarray: A NumPy array with the revealed values.
    """
    if len(array.shape) == 1:
        return np.array([v.to_public() for v in array])
    return np.array([reveal_array(array[i]) for i in range(array.shape[0])])


def modular_inverse(value, modulo=PRIME):
    return pow(value, modulo - 2, modulo)


def gauss_jordan_Zn(matrix):
    """
    Perform Gauss-Jordan elimination on a given matrix.

    Parameters:
    matrix (numpy.ndarray): The input matrix to perform Gauss-Jordan elimination on.

    Returns:
    numpy.ndarray: The reduced row echelon form of the input matrix.
    """
    # Perform Gauss-Jordan elimination over Zn using numpy arrays

    # Make a copy of the matrix to avoid modifying the original
    mat = [row[:] for row in matrix]
    rows = len(mat)
    cols = len(mat[0])

    # Forward elimination
    for i in range(rows):
        # Find pivot row
        pivot_row = i
        while pivot_row < rows and mat[pivot_row][i] == 0:
            pivot_row += 1

        print("PIVOT ROW:", pivot_row)
        # If no pivot found, move to the next column
        if pivot_row == rows:
            continue

        # Swap pivot row with current row
        mat[i], mat[pivot_row] = mat[pivot_row], mat[i]

        # Scale pivot row to have leading 1
        diagonal_element = mat[i][i]
        pivot_inv = modular_inverse(diagonal_element, PRIME)

        # Checks that the inverse is correctly computed
        assert (
            (pivot_inv * diagonal_element) % PRIME
        ) == 1, "Inverse is not reducing to 1"

        for j in range(cols):
            mat[i][j] = (mat[i][j] * pivot_inv) % PRIME

        print(f"PIVOT [{i}]")
        print_matrix(mat)
        # Perform row operations to eliminate entries below pivot
        for j in range(i + 1, rows):
            factor = mat[j][i]
            for k in range(cols):
                mat[j][k] -= (mat[i][k] * factor) % PRIME
                mat[j][k] %= PRIME

    # Backward elimination
    for i in range(rows - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = mat[j][i]
            for k in range(cols):
                mat[j][k] -= (mat[i][k] * factor) % PRIME
                mat[j][k] %= PRIME

    print(
        f"INV:",
    )
    print_matrix(mat)

    return mat


def print_matrix(matrix):
    for row in matrix:
        print("\t", row)


def check_inverse(matrix, inverse):
    result = matrix_multiply(matrix, inverse)
    for i in range(len(result)):
        for j in range(len(result[0])):
            assert (i == j and result[i][j] == 1) or (
                i != j and result[i][j] == 0
            ), f"Position [{i}, {j}] value is: {result[i][j]}"


def test_modular_inverse(a):
    return modular_inverse(a, PRIME) * a % PRIME


def test_gauss_jordan():
    # Example usage
    matrix = [[2, 4, 6, 1, 0, 0], [1, 3, 5, 0, 1, 0], [3, 1, 2, 0, 0, 1]]

    result = gauss_jordan_Zn(matrix)
    initial = [r[:3] for r in matrix]
    inverse = [r[3:] for r in result]
    check_inverse(initial, inverse)
    print("IDENTITY?: ", matrix_multiply(initial, inverse))


def matrix_multiply(A, B):
    # Check if dimensions are compatible for multiplication
    if len(A[0]) != len(B):
        raise ValueError(
            "Number of columns in the first matrix must equal the number of rows in the second matrix."
        )

    # Initialize result matrix with zeros
    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]

    # Perform multiplication
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j] % PRIME
                result[i][j] %= PRIME

    return result


assert test_modular_inverse(1234) == 1
assert test_modular_inverse(1231251) == 1

test_gauss_jordan()

PRIME BITS:  63
PRIME:  18446744072637906947
PIVOT ROW: 0
PIVOT [0]
	 [1, 2, 3, 9223372036318953474, 0, 0]
	 [1, 3, 5, 0, 1, 0]
	 [3, 1, 2, 0, 0, 1]
PIVOT ROW: 1
PIVOT [1]
	 [1, 2, 3, 9223372036318953474, 0, 0]
	 [0, 1, 2, 9223372036318953473, 1, 0]
	 [0, 18446744072637906942, 18446744072637906940, 9223372036318953472, 0, 1]
PIVOT ROW: 2
PIVOT [2]
	 [1, 2, 3, 9223372036318953474, 0, 0]
	 [0, 1, 2, 9223372036318953473, 1, 0]
	 [0, 0, 1, 12297829381758604630, 12297829381758604633, 6148914690879302316]
INV:
	 [1, 0, 0, 3074457345439651158, 12297829381758604631, 6148914690879302316]
	 [0, 1, 0, 3074457345439651160, 12297829381758604629, 6148914690879302315]
	 [0, 0, 1, 12297829381758604630, 12297829381758604633, 6148914690879302316]
IDENTITY?:  [[1, 0, 0], [0, 1, 0], [0, 0, 1]]


In [15]:
LOG_SCALE = 16
SCALE = 1 << LOG_SCALE

In [30]:
matrix = [[2, 4, 6], [1, 3, 5], [3, 1, 2]]

matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 10]]

gauss_jordan_Zn(matrix)

PIVOT ROW: 0
PIVOT [0]
	 [1, 2, 3]
	 [4, 5, 6]
	 [7, 8, 10]
PIVOT ROW: 1
PIVOT [1]
	 [1, 2, 3]
	 [0, 1, 2]
	 [0, 18446744072637906941, 18446744072637906936]
PIVOT ROW: 2
PIVOT [2]
	 [1, 2, 3]
	 [0, 1, 2]
	 [0, 0, 1]
INV:
	 [1, 0, 0]
	 [0, 1, 0]
	 [0, 0, 1]


[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [24]:
np.array(matrix).shape

(3, 3)

In [25]:
def random_lu_matrix(n: int):
    upper = create_random_upper_triangular_matrix(n)
    print(upper)
    lower = create_random_lower_triangular_matrix(n)
    print(lower)
    # Matrix multiplication without using NumPy
    U = matrix_multiply(upper, lower)
    # Calculate determinant of upper triangular matrix
    detU = 1
    for i in range(n):
        detU *= upper[i][i]
        detU %= PRIME
    return U, detU


def random_matrix(n: int):
    L = PRIME >> 1
    return [[random.randint(-L, L - 1) for _ in range(n)] for _ in range(n)], 0


A = np.array([[2.5, 1.1, -1.0], [-3.0, -1.1, 2.2], [-2.3, 1.3, 2.4]]) * SCALE
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
# A =  [[2, 1, -1], [-3, -1, 2], [-2, 1, 2]]
A = A.astype(int).tolist()
# A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) * SCALE
R, detR = random_matrix(3)
print("A: ")
print_matrix(A)
print("R: ")
print_matrix(R)

A: 
	 [163840, 72089, -65536]
	 [-196608, -72089, 144179]
	 [-150732, 85196, 157286]
R: 
	 [-1978528891726524168, 2706279720533992339, 1773758545953940671]
	 [182894720956197415, 3703595317841295797, 225028816964200400]
	 [7040723126779443276, 6629697420826912001, 184626830228237155]


In [26]:
RA = matrix_multiply(R, A)
RA

[[9038144497279356313, 1228070648621609245, 4707868386921357800],
 [4337947527296636760, 10354365186336384242, 1544504659060730071],
 [5734343510062234117, 17802613992944465129, 18272670818374582554]]

In [28]:
# A_inv = gauss_jordan_Zn(np.hstack((RA, R)).tolist())
# A_inv = gauss_jordan_Zn(np.hstack((A, np.identity(3).astype(int))).tolist())
def matrix_concatenate(matrix1, matrix2):
    # Concatenate two matrices horizontally
    concatenated_matrix = []
    for i in range(len(matrix1)):
        concatenated_matrix.append(matrix1[i] + matrix2[i])
    return concatenated_matrix


def identity_matrix(n):
    # Generate identity matrix of size n
    identity = [[0] * n for _ in range(n)]
    for i in range(n):
        identity[i][i] = 1
    return identity


# Concatenate A and identity matrix horizontally and compute its inverse
concatenated_matrix2 = matrix_concatenate(A, identity_matrix(3))
print("CONC:")
print_matrix(concatenated_matrix2)
A_inv = gauss_jordan_Zn(concatenated_matrix2)
A_inv = [r[3:] for r in A_inv]
print("A_INV:")
print_matrix(A_inv)
check_inverse(A, A_inv)

CONC:
	 [163840, 72089, -65536, 1, 0, 0]
	 [-196608, -72089, 144179, 0, 1, 0]
	 [-150732, 85196, 157286, 0, 0, 1]
PIVOT ROW: 0
PIVOT [0]
	 [1, 16971027064825009936, 3689348814527581389, 17216923604465153910, 0, 0]
	 [-196608, -72089, 144179, 0, 1, 0]
	 [-150732, 85196, 157286, 0, 0, 1]
PIVOT ROW: 1
PIVOT [1]
	 [1, 16971027064825009936, 3689348814527581389, 17216923604465153910, 0, 0]
	 [0, 1, 6561492204227846277, 9200214129584587196, 1517930417107853681, 0]
	 [0, 11510750286927697017, 7378697629055259772, 16479121395554044266, 0, 1]
PIVOT ROW: 2
PIVOT [2]
	 [1, 16971027064825009936, 3689348814527581389, 17216923604465153910, 0, 0]
	 [0, 1, 6561492204227846277, 9200214129584587196, 1517930417107853681, 0]
	 [0, 0, 1, 4502069005115998434, 8296625025117853194, 8634205766202028276]
INV:
	 [1, 0, 0, 5279584341320691167, 2906820179692690234, 13198777962789707070]
	 [0, 1, 0, 3242985501618919338, 870221339990918405, 4993705346321639511]
	 [0, 0, 1, 4502069005115998434, 8296625025117853194, 86

In [29]:
# Concatenate RA and R horizontally and compute its inverse
concatenated_matrix1 = matrix_concatenate(RA, R)
print("CONC:")
print_matrix(concatenated_matrix1)
A_inv = gauss_jordan_Zn(concatenated_matrix1)
A_inv = [r[3:] for r in A_inv]
print("A_INV:")
print_matrix(A_inv)
check_inverse(A, A_inv)

CONC:
	 [9038144497279356313, 1228070648621609245, 4707868386921357800, -1978528891726524168, 2706279720533992339, 1773758545953940671]
	 [4337947527296636760, 10354365186336384242, 1544504659060730071, 182894720956197415, 3703595317841295797, 225028816964200400]
	 [5734343510062234117, 17802613992944465129, 18272670818374582554, 7040723126779443276, 6629697420826912001, 184626830228237155]
PIVOT ROW: 0
PIVOT [0]
	 [1, 14709088135397864018, 9922494248549961216, 8559304890794031747, 8554166415944737854, 5879356522143228507]
	 [4337947527296636760, 10354365186336384242, 1544504659060730071, 182894720956197415, 3703595317841295797, 225028816964200400]
	 [5734343510062234117, 17802613992944465129, 18272670818374582554, 7040723126779443276, 6629697420826912001, 184626830228237155]
PIVOT ROW: 1
PIVOT [1]
	 [1, 14709088135397864018, 9922494248549961216, 8559304890794031747, 8554166415944737854, 5879356522143228507]
	 [0, 1, 17666381608181099210, 16245033911133029330, 16428447888378145543, 148

In [22]:
import matplotlib.pyplot as plt

d = {
    17: 0.572,
    18: 0.984,
    19: 1.909,
    20: 3.705,
    21: 7.318,
    22: 14.276,
    23: 28.548,
    24: 60.391,
    25: 120.645,
}

# Sorting the dictionary by keys
lists = sorted(d.items())

# Unpacking the sorted lists into two tuples
x, y = zip(*lists)

# Plotting the data
plt.figure(figsize=(10, 6))  # Adjusting figure size for better visibility
plt.plot(
    x, y, marker="o", linestyle="-", color="b"
)  # Adding markers, line style, and color
plt.title(
    "Consecutive Multiplications vs Compilation Time (nada build)"
)  # Adding title
plt.xlabel("Number of Multiplications")  # Adding label to x-axis
plt.ylabel("Time (seconds)")  # Adding label to y-axis
plt.grid(True)  # Adding grid for better visualization
plt.legend()  # Adding legend
plt.tight_layout()  # Adjusting layout for better spacing
plt.show()  # Displaying the plot

ModuleNotFoundError: No module named 'matplotlib'